In [39]:
import datetime
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import json

tjson=[]

#Set search parameters for model subject tweets to be webscraped.
firsttime = dt.now()

year=2013
disease="adhd"
startdate = datetime.date(2013, 2, 1)
enddate = datetime.date(2018, 4, 1)
daterange = [startdate]

while startdate < enddate:
    if not startdate.month % 12:
        startdate = datetime.date(startdate.year+1, 1, 1)
    else:
        startdate = datetime.date(startdate.year, startdate.month+1, 1)
    daterange.append(startdate)

# querylist=[
#     "I was recently diagnosed with "+disease,
#     "I was diagnosed with "+disease
# ]
    
    
querylist=[ "my nutritionist",
           "my foodstamps"

]

labellist=[0,1]

newrange = daterange[0::2]

for index,searchdate in enumerate(newrange[:-1]):
    start = newrange[index].strftime('%Y-%-m-%d')
    if newrange[index].month==12:
        newmonth=1
    else:
        newmonth=newrange[index].month+1
    end = datetime.date(newrange[index].year, newmonth, 1).strftime('%Y-%-m-%d')
    for label,query in zip(labellist,querylist):
        #Query tweets from search page
        print
        print('Collecting "%s" tweets from %s to %s at %s'%(query,start,end,dt.now()))
        print
        startTime = dt.now()
        q=query.replace(' ','%20')
        prefix='https://twitter.com/search?f=tweets&vertical=default&q=%22'
        browser = webdriver.Chrome('./chromedriver')
        #browser = webdriver.Firefox()
        #browser = webdriver.Safari()
        browser.get(prefix+q+'%22%20since%3A'+start+'%20until%3A'+end+'&src=typd')

        #Scroll to bottom of search results page to webscrape all results.
        while True:
            elemsCount = browser.execute_script("return document.querySelectorAll('.stream-items > li.stream-item').length")
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            try:
                WebDriverWait(browser, 2).until(lambda x: x.find_element_by_xpath(
                    "//*[contains(@class,'stream-items')]/li[contains(@class,'stream-item')]["+str(elemsCount+1)+"]"))
            except:
                break  

        #Get lists of all tweets, usernames, and dates
        tweets = browser.find_elements_by_class_name('js-tweet-text')
        tweetids = browser.find_elements_by_class_name('original-tweet')
        users = browser.find_elements_by_css_selector('.original-tweet')
        dates = browser.find_elements_by_class_name('tweet-timestamp')

        twtcnt=0

        #Loop through tweet info and assign to dict
        for user,date,tweetid,tweet in zip(users,dates,tweetids,tweets):
            tdata={}
            _tweetid=tweetid.get_attribute("data-tweet-id")
            _tweetdate=date.get_attribute('title')
            _user=user.get_attribute('data-screen-name')
            _tweettext=tweet.text#.encode('utf-8').decode("utf-8").encode("ascii","ignore")
            tdata['username']=_user
            tdata['key_phrase']=query
            tdata['marker_tweet']=_tweettext
            tdata['marker_tweet_id']=_tweetid
            tdata['marker_tweet_date']=_tweetdate
            tdata['label']=label

            if any(['bot' in _user]):
                continue
                

            twtcnt+=1

            tjson.append(tdata)


            if twtcnt % 100 == 0:
                print('-----%s tweets collected at %s.'%(twtcnt,dt.now()))
        browser.quit()
        print('-----%s "%s" tweets collected in %s seconds.'%(twtcnt,query,dt.now()-startTime))
    
print
print
print
print('Total Time: %s'%(dt.now()-firsttime))

#Write dicts to json file.
with open('food_risk_train_set2.json', 'w') as outfile:
    outfile.write(json.dumps(tjson))

-----100 tweets collected at 2018-04-16 23:38:15.199393.
-----200 tweets collected at 2018-04-16 23:38:20.909346.
-----300 tweets collected at 2018-04-16 23:38:26.705709.
-----400 tweets collected at 2018-04-16 23:38:32.510753.
-----500 tweets collected at 2018-04-16 23:38:38.369922.
-----600 tweets collected at 2018-04-16 23:38:44.406367.
-----642 "my nutritionist" tweets collected in 0:01:11.983078 seconds.
-----100 tweets collected at 2018-04-16 23:39:21.298419.
-----200 tweets collected at 2018-04-16 23:39:26.493467.
-----300 tweets collected at 2018-04-16 23:39:31.840777.
-----400 tweets collected at 2018-04-16 23:39:37.157161.
-----500 tweets collected at 2018-04-16 23:39:42.614104.
-----574 "my foodstamps" tweets collected in 0:00:59.617022 seconds.
-----100 tweets collected at 2018-04-16 23:40:13.034712.
-----200 tweets collected at 2018-04-16 23:40:19.699794.
-----300 tweets collected at 2018-04-16 23:40:28.223374.
-----339 "my nutritionist" tweets collected in 0:00:44.541319 

-----200 tweets collected at 2018-04-16 23:54:58.675073.
-----300 tweets collected at 2018-04-16 23:55:04.452808.
-----400 tweets collected at 2018-04-16 23:55:10.523930.
-----476 "my nutritionist" tweets collected in 0:00:53.842228 seconds.
-----100 tweets collected at 2018-04-16 23:55:30.770030.
-----139 "my foodstamps" tweets collected in 0:00:17.484548 seconds.
-----100 tweets collected at 2018-04-16 23:56:00.382109.
-----200 tweets collected at 2018-04-16 23:56:05.834413.
-----300 tweets collected at 2018-04-16 23:56:11.441099.
-----352 "my nutritionist" tweets collected in 0:00:41.700890 seconds.
-----100 tweets collected at 2018-04-16 23:56:31.715200.
-----152 "my foodstamps" tweets collected in 0:00:19.760969 seconds.
-----100 tweets collected at 2018-04-16 23:56:56.769156.
-----200 tweets collected at 2018-04-16 23:57:02.169880.
-----292 "my nutritionist" tweets collected in 0:00:32.751029 seconds.
-----100 tweets collected at 2018-04-16 23:57:22.190027.
-----119 "my foodstamp

-----320 "my nutritionist" tweets collected in 0:00:40.874630 seconds.
-----66 "my foodstamps" tweets collected in 0:00:11.363407 seconds.
Total Time: 0:30:36.027329


In [ ]:
#steps to remove retweets and tweets from duplicate accounts
import pandas as pd
import numpy as np


d = pd.read_json('food_risk_train_set2.json')
print(d.groupby('label')['marker_tweet'].count())


d['upper_tweet']=d.marker_tweet.str.upper()

d = d[d.upper_tweet.str.contains('"') == False]

d = d[d.marker_tweet.str.contains("RT") == False]


dedup1 = d.drop_duplicates(['username','marker_tweet'],False)
dedup2 = dedup1.sort_values('marker_tweet_id').drop_duplicates(['username'],keep='last')

dedup2.to_json(path_or_buf='dedup_food_train_set_4_16.json',orient='records')